# Gender representation analysis of Stack Overflow developer survey 2021


Since 2011, Stack Overflow Annual Developer Survey tries to capture the current state of professional and enthusiast programmers' community by covering a range of questions regarding the education, experience, favourite programming software etc.

This project is focused on analysing the demographic aspect of the survey, specifically the gender representation and some of the main distinction areas between genders within the coding realm.

Three **key questions** this project will focus on are:
>**1.** What is the profile of gender representation in 2021 and are there any identifiable trends when compared to the surveys from the 2017-2021 period?
>
>**2.** Is the engagement of different genders in various coding-related communities the same of does it vary?
>
>**3.** Do different genders develop interest in coding at the same age or is there a noticeable difference?      

#### Setting up

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import seaborn as sns
import re
%matplotlib inline

#uncomment if on laptop 1
#df = pd.read_csv('./Stackoverflow survey 2021/survey_results_public.csv')
#schema = pd.read_csv('./Stackoverflow survey 2021/survey_results_schema.csv')

#uncomment if on laptop 2
df = pd.read_csv('./stack-overflow-developer-survey-2021/survey_results_public.csv')
schema = pd.read_csv('./stack-overflow-developer-survey-2021/survey_results_schema.csv')

In [2]:
schema.head(5) 

qid       qname                                           question  \
0  QID16          S0  <div><span style="font-size:19px;"><strong>Hel...   
1  QID12    MetaInfo                                  Browser Meta Info   
2   QID1          S1  <span style="font-size:22px; font-family: aria...   
3   QID2  MainBranch  Which of the following options best describes ...   
4  QID24  Employment  Which of the following best describes your cur...   

   force_resp  type selector  
0       False    DB       TB  
1       False  Meta  Browser  
2       False    DB       TB  
3        True    MC     SAVR  
4       False    MC     MAVR

In [3]:
df.head(5)

ResponseId                                         MainBranch  \
0           1                     I am a developer by profession   
1           2             I am a student who is learning to code   
2           3  I am not primarily a developer, but I write co...   
3           4                     I am a developer by profession   
4           5                     I am a developer by profession   

                                          Employment  \
0  Independent contractor, freelancer, or self-em...   
1                                 Student, full-time   
2                                 Student, full-time   
3                                 Employed full-time   
4  Independent contractor, freelancer, or self-em...   

                                             Country US_State UK_Country  \
0                                           Slovakia      NaN        NaN   
1                                        Netherlands      NaN        NaN   
2                                 Russian Federation      NaN        NaN   
3                                            Austria      NaN        NaN   
4  United Kingdom of Great Britain and Northern I...      NaN    England   

                                             EdLevel     Age1stCode  \
0  Secondary school (e.g. American high school, G...  18 - 24 years   
1       Bachelor’s degree (B.A., B.S., B.Eng., etc.)  11 - 17 years   
2       Bachelor’s degree (B.A., B.S., B.Eng., etc.)  11 - 17 years   
3    Master’s degree (M.A., M.S., M.Eng., MBA, etc.)  11 - 17 years   
4    Master’s degree (M.A., M.S., M.Eng., MBA, etc.)   5 - 10 years   

                                           LearnCode YearsCode  \
0  Coding Bootcamp;Other online resources (ex: vi...       NaN   
1  Other online resources (ex: videos, blogs, etc...         7   
2  Other online resources (ex: videos, blogs, etc...       NaN   
3                                                NaN       NaN   
4                            Friend or family member        17   

          ...                      Age Gender Trans                Sexuality  \
0         ...          25-34 years old    Man    No  Straight / Heterosexual   
1         ...          18-24 years old    Man    No  Straight / Heterosexual   
2         ...          18-24 years old    Man    No        Prefer not to say   
3         ...          35-44 years old    Man    No  Straight / Heterosexual   
4         ...          25-34 years old    Man    No                      NaN   

                      Ethnicity                Accessibility  \
0  White or of European descent            None of the above   
1  White or of European descent            None of the above   
2             Prefer not to say            None of the above   
3  White or of European descent  I am deaf / hard of hearing   
4  White or of European descent            None of the above   

        MentalHealth           SurveyLength                  SurveyEase  \
0  None of the above  Appropriate in length                        Easy   
1  None of the above  Appropriate in length                        Easy   
2  None of the above  Appropriate in length                        Easy   
3                NaN  Appropriate in length  Neither easy nor difficult   
4                NaN  Appropriate in length                        Easy   

  ConvertedCompYearly  
0             62268.0  
1                 NaN  
2                 NaN  
3                 NaN  
4                 NaN  

[5 rows x 48 columns]



File 'survey_results_schema.csv' contains a full list of questions and the name of column within the file with survey results 'survey_results_public.csv'.

After previewing first couple of entries in both **schema** and the main **df** dataframe, it is evident the 'question' column within schema contains entries with additional formatting snippets of code such as '< div >< span style="font-size:19px;">< strong> ' which makes it hard to read and could use a bit of cleaning.

Two functions below are to remove all formatting tags and to callout the full question when given the column name.

In [4]:
def remove_tags(string): #removes formatting tags from strings
    '''
    INPUT - string - string containg formatting tags
    OUTPUT - 
            result - string without any formatting tags
    '''
    #patterns =['<.*?>','\n','*']
    result = re.sub('<.*?>|\n|\*','',string)
    #result = re.sub('<.*?>|\n','',string)   
    return result

schema['question_cleaned']=schema['question'].apply(lambda cw : remove_tags(cw))


def get_description(column_name, schema=schema):
    '''
    INPUT - column_name - string, the name of the column you would like to know about
            schema - pandas dataframe with the schema of the developers survey
            
    OUTPUT - desc - string, the description of the column
    '''
    desc = list(schema[schema['qname'] == column_name]['question_cleaned'])#[0]
    return desc

In [5]:
pd.options.display.max_colwidth = 120
schema[['qname','question_cleaned']][3:].head(5)

qname  \
3  MainBranch   
4  Employment   
5     Country   
6    US_State   
7  UK_Country   

                                                                                                    question_cleaned  
3  Which of the following options best describes you today? Here, by "developer" we mean "someone who writes code."   
4                                              Which of the following best describes your current employment status?  
5                                                                                                Where do you live?   
6                         In which state or territory of the USA do you live? This information will be kept private.  
7                            In which part of the United Kingdom do you live? This information will be kept private.

In [6]:
get_description('Employment')

['Which of the following best describes your current employment status?']

#### Cleaning the data

In [7]:
df.columns #looking at the 

Index(['ResponseId', 'MainBranch', 'Employment', 'Country', 'US_State',
       'UK_Country', 'EdLevel', 'Age1stCode', 'LearnCode', 'YearsCode',
       'YearsCodePro', 'DevType', 'OrgSize', 'Currency', 'CompTotal',
       'CompFreq', 'LanguageHaveWorkedWith', 'LanguageWantToWorkWith',
       'DatabaseHaveWorkedWith', 'DatabaseWantToWorkWith',
       'PlatformHaveWorkedWith', 'PlatformWantToWorkWith',
       'WebframeHaveWorkedWith', 'WebframeWantToWorkWith',
       'MiscTechHaveWorkedWith', 'MiscTechWantToWorkWith',
       'ToolsTechHaveWorkedWith', 'ToolsTechWantToWorkWith',
       'NEWCollabToolsHaveWorkedWith', 'NEWCollabToolsWantToWorkWith', 'OpSys',
       'NEWStuck', 'NEWSOSites', 'SOVisitFreq', 'SOAccount', 'SOPartFreq',
       'SOComm', 'NEWOtherComms', 'Age', 'Gender', 'Trans', 'Sexuality',
       'Ethnicity', 'Accessibility', 'MentalHealth', 'SurveyLength',
       'SurveyEase', 'ConvertedCompYearly'],
      dtype='object')

For the key questions outlined at the start of this notebook, the survey columns worth focusing on are **Gender, Age, Age1stCode, YearsCode, SOComm and NEWOtherComms**. 

The missing values in 'Gender' column can be filled in with the 'Prefer not to say' as there is no information on the gender of the respondent. With this approach, the sample size remains unchanged, and the NaN values will be accounted for when calculating gender representation.

Looking at the list of answers for Gender and YearsCode, it is clear the survey questions were formatted as multiple choice (Gender) and free-form answer (YearsCode) questions.

Two functions below are defined to group different answers into corresponding category/range.

In [8]:
df['Gender'].isna().sum()

1153

In [9]:
df['Gender'].fillna('Prefer not to say', inplace=True)
df['Gender'].value_counts()

Man                                                                                   74817
Woman                                                                                  4120
Prefer not to say                                                                      2595
Non-binary, genderqueer, or gender non-conforming                                       690
Or, in your own words:                                                                  413
Man;Or, in your own words:                                                              268
Man;Non-binary, genderqueer, or gender non-conforming                                   252
Woman;Non-binary, genderqueer, or gender non-conforming                                 147
Man;Woman                                                                                41
Non-binary, genderqueer, or gender non-conforming;Or, in your own words:                 21
Man;Woman;Non-binary, genderqueer, or gender non-conforming                     

In [10]:
df['YearsCode'].value_counts()

5                     6190
10                    6067
6                     5565
4                     5049
7                     4940
8                     4714
3                     4449
15                    3625
20                    3419
2                     3342
12                    3188
9                     3009
11                    2344
13                    2016
25                    1984
14                    1813
1                     1695
30                    1660
16                    1491
Less than 1 year      1489
18                    1386
17                    1331
35                    1007
22                     941
21                     863
40                     830
23                     811
19                     659
24                     639
26                     533
27                     492
28                     455
38                     361
32                     331
37                     329
33                     309
34                     268
3

In [11]:
def gender_iden(gender_str):
    '''
    INPUT
        gender_str - a string of one of the values from the Gender column
    
    OUTPUT
        return 0 if the string is  in ("Prefer not to say")
        return 1 if the string is  in ("Men")
        return 2 if the string is  in ("Women")
        
        return 3 otherwise, this case will group toghther all answers that contain 'Non-binary, genderqueer, or gender non-conforming' along with additional user selection
    
    '''
    if gender_str=="Prefer not to say":
        return 0
    elif gender_str=="Man" or gender_str== "Male":
        return 1
    elif gender_str=="Woman" or gender_str=="Female":
        return 2
    else:
        return 3


def years_code_grp(yearscode_str):
    '''
    INPUT
        yearscode_str - a string of one of the values from the YearsCode column
    
    OUTPUT
        return one of the values 'Less than 1 year', '1 - 5 years', '6 - 10 years', '11 - 20 years', '21 - 50 years','More than 50 years'
    
    '''

    try:
        years=int(yearscode_str)
        if years in range(1,6): res='1 - 5 years'
        elif years in range(6,11): res='6 - 10 years'
        elif years in range(11,21): res='11 - 20 years'    
        else: res='21 - 50 years'
        return res
    except: return yearscode_str

Creating new columns within the main **df** dataframe that correspond to the grouped values from Gender and YearsCode.

In [12]:
df['GenderIdentity'] = df['Gender'].apply(gender_iden)
df['GenderIdentity'].value_counts()

1    74817
2     4120
0     2595
3     1907
Name: GenderIdentity, dtype: int64

In [13]:
df['YearsCodeGrouped'] = df['YearsCode'].apply(years_code_grp)
df.YearsCodeGrouped.value_counts()

6 - 10 years          24295
11 - 20 years         21272
1 - 5 years           20725
21 - 50 years         13690
Less than 1 year       1489
More than 50 years      170
Name: YearsCodeGrouped, dtype: int64

### 1. Gender representation

In [14]:
df['GenderIdentity'].value_counts()/df.shape[0]*100

1    89.666703
2     4.937739
0     3.110056
3     2.285502
Name: GenderIdentity, dtype: float64

Based on the above calculation, **89.7%** of survey respondents identify as Men, **4.9%** as Women and **3.1%** and Non-binary, genderqueer, or gender non-conforming.

Unsurprisingly, men make up most of the coding world (that is if we assume the survey captured the representative sample of the wider coding community), but has the gender representation changed over the years?

To answer this question, let us look at the gender distribution of survey respondents for the 2017-2021 period. 

In [15]:
df_full=pd.DataFrame()
for year in range(2017,2022):
    df_temp = pd.read_csv('./stack-overflow-developer-survey-'+str(year)+'/survey_results_public.csv',low_memory=False)
    #schema = pd.read_csv('./stack-overflow-developer-survey-'+str(year)+'/survey_results_schema.csv',low_memory=False)
    df_temp['Year']=year
    df_temp['YearRespCount']=df_temp.shape[0]
    df_full=df_full.append(df_temp[['Year','YearRespCount','Gender']], ignore_index=True)

The function above reads in the survey results file for the corresponding value of the **year** variable and appends only the **Gender** column to the main dataframe **df_full** that will be used in the gender representation analysis. To be able to distinguish responses from different years, an additional column called **Year** is added to the dataframe.

Next, the missing data in the Gender column is replaced by the 'Prefer not to say' value and mapped to the corresponding GenderIdentity code.

In [16]:
df_full['Gender'].isna().sum()

69265

In [17]:
df_full['Gender'].fillna('Prefer not to say', inplace=True)
#df_full['Gender'].value_counts()

In [18]:
df_full['GenderIdentity'] = df_full['Gender'].apply(gender_iden)
df_full['GenderIdentity'].value_counts()

1    289796
0     70707
2     20933
3      5594
Name: GenderIdentity, dtype: int64

In [19]:
df_full.head(5)

Year  YearRespCount             Gender  GenderIdentity
0  2017          51392               Male               1
1  2017          51392               Male               1
2  2017          51392               Male               1
3  2017          51392               Male               1
4  2017          51392  Prefer not to say               0

For easy comparison of gender representation of survey respondents over the years, **df_full_grp** dataframe is grouped by Year and GenderIdentity and a new column 'GenderIdentityPerc' represents a percentage of respondents who identify themselves with a certain gender identity.

In [20]:
df_full_grp=df_full.groupby(['Year','YearRespCount','GenderIdentity']).count().reset_index()
df_full_grp.rename(columns={'Gender':'GenderIdentityCount'}, inplace=True)
df_full_grp['GenderIdentityPerc']=df_full_grp['GenderIdentityCount']/df_full_grp['YearRespCount']
df_full_grp.style.format({'GenderIdentityPerc': '{:,.2%}'.format,})

Pivoting the table for a clearer view of the data and plotting the line charts to visualise the results.

In [21]:
df_full_grpiv=df_full_grp.pivot_table(index='Year', columns='GenderIdentity',values='GenderIdentityPerc').reset_index()
df_full_grpiv.columns=['Year','_', 'Men','Women','NB-GQ-NC-TG' ]
#df_full_grpiv.rename(columns={'Year':'Year','0':'NDA','1':'Men', '2':'Women', '3':'NB-GQ-NC-TG' }, inplace=True)
#df_full_grpiv.set_index('Year', inplace=True)
df_full_grpiv.style.format({'_': '{:,.2%}'.format,'Men': '{:,.2%}'.format,'Women': '{:,.2%}'.format,'NB-GQ-NC-TG': '{:,.2%}'.format,})
#type(df_full_grpiv)
#df_full_grpiv

In [22]:
g=sns.catplot(x='Year', y='GenderIdentityPerc', hue='GenderIdentity', data=df_full_grp[df_full_grp['GenderIdentity']!=0], kind='point', height=4.8, aspect=1.5, palette='Set2', facet_kws={'legend_out':True})

labels_lst=['Men','Women','NB-GQ-NC-TG']
for t,l in zip(g._legend.texts, labels_lst):
    t.set_text(l)
    
    #plt.legend(bbox_to_anchor=(1.02, 1),loc='upper left', title=False, frameon=False)#.get_texts()[0].set_text('Men')   
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))


Based on the above graph, **the participation of female as well as NB-GQ-NC-TG (non-binary, gender queer, non-conforming and transgender) coders in the Stack Overflow's annual developer's survey has been stagnating**. Yearly changes cannot be described as either significant increase or decreases in representation as they oscillate in the range of  <span>&#177;</span>1.5%.

The only significant yearly changes are in the participation of male survey respondents. When looking into the survey entries without disclosed gender (entries with missing values and 'Prefer not to say') shown in the graph below, a direct correlation between the two groups is identifiable.

In [23]:
g=sns.catplot(x='Year', y='GenderIdentityPerc', hue='GenderIdentity', data=df_full_grp[df_full_grp['GenderIdentity']<2], kind='point', height=4.8, aspect=1.5,palette='pastel', facet_kws={'legend_out':True})

labels_lst2=['Not disclosed','Men']
for t,l in zip(g._legend.texts, labels_lst2):
    t.set_text(l)
    
#plt.legend(bbox_to_anchor=(1.02, 0.5),loc='upper left',title='GenderIdentity', frameon=False).get_texts()[0].set_text('Prefer not to say')
plt.gca().yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))

### 2. Activity within coding community

The objective of this section is to investigate engagement rates of different genders within Stack Overflow's and other online communities focused on software programming.

The function below counts the number of entries within a specified column of a dataframe and is applied to the dataframe subset that corresponds to a certain gender identity.

In [24]:
gnd_0 = df[df['GenderIdentity'] == 0] # Subset df to only those with GenderIdentity of 0 (Prefer not to say)
gnd_m = df[df['GenderIdentity'] == 1] # Subset df to only those with GenderIdentity of 1 (Man)
gnd_w = df[df['GenderIdentity'] == 2] # Subset df to only those with GenderIdentity of 2 (Woman)
gnd_nb = df[df['GenderIdentity'] == 3] # Subset df to only those with GenderIdentity of 3 (Nonbinary)


In [25]:
def counter(df, col):
    '''
    INPUT 
        df - a dataframe holding the SOCOmm column
        col - name of a column to explore
    OUTPUT
        counter - a dataframe with the count of how many individuals
    '''
    counter = df[col].value_counts().reset_index()
    counter[col]=counter[col]/df.shape[0]
    counter.rename(columns={'index': 'answer', col: 'perc'}, inplace=True)
    
    return counter
    

In [26]:
gnd_0_perc = counter(gnd_0, 'SOComm')
gnd_m_perc = counter(gnd_m, 'SOComm')
gnd_w_perc = counter(gnd_w, 'SOComm')
gnd_nb_perc = counter(gnd_nb, 'SOComm')


comp_df = pd.merge(gnd_m_perc, gnd_w_perc, left_on='answer', right_on='answer')
comp_df.columns = ['SOMember','Men', 'Women']
comp_df['GenderDiff'] = comp_df['Men'] - comp_df['Women']
comp_df.sort_values(by=['GenderDiff'], inplace=True, ascending=False)
comp_df.style.bar(subset=['GenderDiff'], align='mid', color=['#fec20c', '#00ab41']).format({'Men': '{:,.2%}'.format,
    'Women': '{:,.2%}'.format,
    'GenderDiff': '{:,.2%}'.format,
})

By comparing the percentages of male and female respondents (column **GenderDiff**), women are less likely to be active members of Stack Overflow community (SOComm).

The same calculation can be applied to respondents who identify themselves as NB, GQ or TGNC (non-binary, gender queer, transgender, or non-conforming). The comparison is shown below with columns **GenderDiff_MW**, **GenderDiff_MNb** and **GenderDiff_MND** corresponding to differences men-women, men-non-binary, gender queer, non-conforming and transgender and men-not disclosed (added only for a reference).

In [27]:
gnd_0_perc = counter(gnd_0, 'SOComm')
gnd_m_perc = counter(gnd_m, 'SOComm')
gnd_w_perc = counter(gnd_w, 'SOComm')
gnd_nb_perc = counter(gnd_nb, 'SOComm')

comp_df = pd.merge(gnd_m_perc, gnd_w_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_nb_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_0_perc, left_on='answer', right_on='answer')

sorter=['Yes, definitely','Yes, somewhat','Neutral','No, not really','No, not at all', 'Not sure']
sorterIndex = dict(zip(sorter, range(len(sorter))))

comp_df.columns = ['SOMember','Men', 'Women', 'Nonbinary', 'NotDisclosed']
comp_df['SOMember_Rank'] = comp_df['SOMember'].map(sorterIndex)

comp_df['GenderDiff_MW'] = comp_df['Men'] - comp_df['Women']
comp_df['GenderDiff_MNb'] = comp_df['Men'] - comp_df['Nonbinary']
comp_df['GenderDiff_MND'] = comp_df['Men'] - comp_df['NotDisclosed']
comp_df.sort_values(['SOMember_Rank'],ascending = True, inplace = True)
comp_df.drop('SOMember_Rank', 1, inplace = True)
comp_df.reset_index(inplace=True, drop=True)

#comp_df.sort_values(by=['GenderDiff_MW'], ignore_index=True, inplace=True, ascending=False)
comp_df.style.bar(subset=['GenderDiff_MW'], align='mid', color=['#fec20c', '#00ab41']).bar(subset=['GenderDiff_MNb'], align='mid', color=['#6a0dad', '#00ab41']).bar(subset=['GenderDiff_MND'], align='mid', color=['#8e8e8e', '#00ab41']).format({'Men': '{:,.2%}'.format,
    'Women': '{:,.2%}'.format,'Nonbinary': '{:,.2%}'.format,'NotDisclosed': '{:,.2%}'.format,'GenderDiff_MW': '{:,.2%}'.format,'GenderDiff_MNb': '{:,.2%}'.format,'GenderDiff_MND': '{:,.2%}'.format,
})

Out of all the gender identity groups, male survey respondents have consistently higher engagement in the SOComm. 

Can the same be said for the engagement within other similar online communities?

In [28]:
df.NEWOtherComms.value_counts()

No     53364
Yes    29464
Name: NEWOtherComms, dtype: int64

In [29]:
gnd_0_perc = counter(gnd_0, 'NEWOtherComms')
gnd_m_perc = counter(gnd_m, 'NEWOtherComms')
gnd_w_perc = counter(gnd_w, 'NEWOtherComms')
gnd_nb_perc = counter(gnd_nb, 'NEWOtherComms')

comp_df = pd.merge(gnd_m_perc, gnd_w_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_nb_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_0_perc, left_on='answer', right_on='answer')

comp_df.columns = ['NEWOtherComms','Men', 'Women', 'Nonbinary', 'NotDisclosed']

comp_df['GenderDiff_MW'] = comp_df['Men'] - comp_df['Women']
comp_df['GenderDiff_MNb'] = comp_df['Men'] - comp_df['Nonbinary']
comp_df['GenderDiff_MND'] = comp_df['Men'] - comp_df['NotDisclosed']

comp_df.sort_values(by=['GenderDiff_MW'], inplace=True, ascending=False)
comp_df.style.bar(subset=['GenderDiff_MW'], align='mid', color=['#fec20c', '#00ab41']).bar(subset=['GenderDiff_MNb'], align='mid', color=['#6a0dad', '#00ab41']).bar(subset=['GenderDiff_MND'], align='mid', color=['#8e8e8e', '#00ab41']).format({'Men': '{:,.2%}'.format,
    'Women': '{:,.2%}'.format,'Nonbinary': '{:,.2%}'.format,'NotDisclosed': '{:,.2%}'.format,'GenderDiff_MW': '{:,.2%}'.format,'GenderDiff_MNb': '{:,.2%}'.format,'GenderDiff_MND': '{:,.2%}'.format,
})

Looks like higher male engagement is not characteristic only for Stack Overflow community.

It would be interesting to see if there are any programming focused communities where women, NB, GQ and TGNC are more likely to join and have higher engagement rates. Unfortunately, the answers to that survey question were not made public by SO.

### 3. Developing interest in coding

The objective of this section is to explore if there are any identifiable trends when it comes to age at which interest in coding occurs within different gender identity groups.

To do that, code below groups each of the data subsets (created based on the gender identity of the survey respondents) by the age group and counts the number of respondents.

In [30]:
df.Age.value_counts()

25-34 years old       32568
18-24 years old       20993
35-44 years old       15183
45-54 years old        5472
Under 18 years old     5376
55-64 years old        1819
Prefer not to say       575
65 years or older       421
Name: Age, dtype: int64

In [31]:
age_m_g=gnd_m.groupby('Age')[['ResponseId']].count().reset_index()
age_m_g.rename(columns={'ResponseId':'AgeGroupCount'}, inplace=True)
age_m_g

Age  AgeGroupCount
0     18-24 years old          18826
1     25-34 years old          29511
2     35-44 years old          14090
3     45-54 years old           5106
4     55-64 years old           1652
5   65 years or older            365
6   Prefer not to say            215
7  Under 18 years old           4793

In [32]:
age_m=gnd_m.groupby(['Age','Age1stCode'])[['ResponseId']].count().reset_index()
age_m.head(5)

Age           Age1stCode  ResponseId
0  18-24 years old        11 - 17 years       11793
1  18-24 years old        18 - 24 years        4957
2  18-24 years old        25 - 34 years          12
3  18-24 years old         5 - 10 years        1800
4  18-24 years old  Older than 64 years           2

Merging two dataframes, **age_m_g** which is grouped based on age group and **age_m** grouped by age of first code, to get  data for both group types in the same view.

In [33]:
# Men
age_m_df=age_m.merge(age_m_g, on='Age')
age_m_df['Age1stCodePerc']=age_m_df['ResponseId']/age_m_df['AgeGroupCount']
sorter=[ 'Younger than 5 years','5 - 10 years','11 - 17 years','18 - 24 years','25 - 34 years', '35 - 44 years', '45 - 54 years', '55 - 64 years', 'Older than 64 years']
sorterIndex = dict(zip(sorter, range(len(sorter))))
age_m_df['Age1stCode_Rank'] = age_m_df['Age1stCode'].map(sorterIndex)
age_m_df.sort_values(['Age1stCode_Rank'],ascending = True, inplace = True)
age_m_df.drop('Age1stCode_Rank', 1, inplace = True)

age_m_df.head(5).style.format({'Age1stCodePerc': '{:,.2%}'.format,})

Pivoting dataframe for a view with more clarity.

In [34]:
age_m_dfp=age_m_df.pivot_table(index='Age', columns='Age1stCode',values='Age1stCodePerc').reset_index()

sorter_age1stcode=[ 'Younger than 5 years','5 - 10 years','11 - 17 years','18 - 24 years','25 - 34 years', '35 - 44 years', '45 - 54 years', '55 - 64 years', 'Older than 64 years']

sorter_age=[ 'Under 18 years old','18-24 years old','25-34 years old', '35-44 years old', '45-54 years old', '55-64 years old',
 '65 years or older', 'Prefer not to say']

#sorterIndex = dict(zip(sorter_age, range(len(sorter_age))))
#age_m_dfp['Age_Rank'] = age_m_dfp['Age'].map(sorterIndex)
#age_m_dfp.sort_values(['Age_Rank'],ascending = True, inplace = True)
#age_m_dfp.drop('Age_Rank', 1, inplace = True)

age_m_dfp.set_index('Age', inplace=True)

age_m_dfp= age_m_dfp.reindex(index=sorter_age)
age_m_dfp= age_m_dfp.reindex(columns=sorter_age1stcode)

age_m_dfp.style.format({'11 - 17 years': '{:,.2%}'.format, '18 - 24 years': '{:,.2%}'.format, '25 - 34 years': '{:,.2%}'.format,
       '35 - 44 years': '{:,.2%}'.format, '45 - 54 years': '{:,.2%}'.format, '5 - 10 years': '{:,.2%}'.format, '55 - 64 years': '{:,.2%}'.format,
       'Older than 64 years': '{:,.2%}'.format, 'Younger than 5 years': '{:,.2%}'.format,})
#age_df_piv.columns

The table above shows percentage of respondents that wrote their first code at a certain are range (columns) within each of the age group (rows). <br> 
**E.g.,** Out of all respondents that are 25-34 years old, 51.52% wrote their first code between the ages of 11 and 17 years.

Below is the same calculation applied to other data subsets.

In [35]:
# Women
age_w_g=gnd_w.groupby('Age')[['ResponseId']].count().reset_index()
age_w_g.rename(columns={'ResponseId':'AgeGroupCount'}, inplace=True)
age_w=gnd_w.groupby(['Age','Age1stCode'])[['ResponseId']].count().reset_index()
age_w_df=age_w.merge(age_w_g, on='Age')
age_w_df['Age1stCodePerc']=age_w_df['ResponseId']/age_w_df['AgeGroupCount']

age_w_dfp=age_w_df.pivot_table(index='Age', columns='Age1stCode',values='Age1stCodePerc').reset_index()
age_w_dfp.set_index('Age', inplace=True)

age_w_dfp= age_w_dfp.reindex(index=sorter_age)
age_w_dfp= age_w_dfp.reindex(columns=sorter_age1stcode)

age_w_dfp.style.format({'11 - 17 years': '{:,.2%}'.format, '18 - 24 years': '{:,.2%}'.format, '25 - 34 years': '{:,.2%}'.format,
       '35 - 44 years': '{:,.2%}'.format, '45 - 54 years': '{:,.2%}'.format, '5 - 10 years': '{:,.2%}'.format, '55 - 64 years': '{:,.2%}'.format,
       'Older than 64 years': '{:,.2%}'.format, 'Younger than 5 years': '{:,.2%}'.format,})

In [36]:
#NB, GQ and TGNC
age_nb_g=gnd_nb.groupby('Age')[['ResponseId']].count().reset_index()
age_nb_g.rename(columns={'ResponseId':'AgeGroupCount'}, inplace=True)
age_nb=gnd_nb.groupby(['Age','Age1stCode'])[['ResponseId']].count().reset_index()
age_nb_df=age_nb.merge(age_nb_g, on='Age')
age_nb_df['Age1stCodePerc']=age_nb_df['ResponseId']/age_nb_df['AgeGroupCount']

age_nb_dfp=age_nb_df.pivot_table(index='Age', columns='Age1stCode',values='Age1stCodePerc').reset_index()
age_nb_dfp.set_index('Age', inplace=True)

age_nb_dfp= age_nb_dfp.reindex(index=sorter_age)
age_nb_dfp= age_nb_dfp.reindex(columns=sorter_age1stcode)

age_nb_dfp.style.format({'11 - 17 years': '{:,.2%}'.format, '18 - 24 years': '{:,.2%}'.format, '25 - 34 years': '{:,.2%}'.format,
       '35 - 44 years': '{:,.2%}'.format, '45 - 54 years': '{:,.2%}'.format, '5 - 10 years': '{:,.2%}'.format, '55 - 64 years': '{:,.2%}'.format,
       'Older than 64 years': '{:,.2%}'.format, 'Younger than 5 years': '{:,.2%}'.format,})

In [37]:
fig, axes = plt.subplots(ncols=3,sharey=True, figsize=(15,5))
sns.heatmap(age_m_dfp, annot=True, fmt='.2f', cmap='Greens', vmin=0, vmax=0.7, ax=axes[0], cbar=False)
sns.heatmap(age_w_dfp, annot=True, fmt='.2f', cmap='Greens', vmin=0, vmax=0.7, ax=axes[1], cbar=False)
sns.heatmap(age_nb_dfp, annot=True, fmt='.2f', cmap='Greens', vmin=0, vmax=0.7, ax=axes[2], cbar=False)
axes[0].title.set_text('Men')
axes[1].title.set_text('Women')
axes[2].title.set_text('NB, GQ and TGNC')
axes[1].set_ylabel('')
axes[2].set_ylabel('');

In this case, the heatmaps above can be used as both visual aid for representation of the data from the previous three tables and as a quick comparison tool of results between different gender identity groups. (It is worth noticing diagonal cut-off line. The upper right area of each heatmap is expectedly empty as for example, respondants of age between 25-34 years could not have written their first code at older age then they are now.)

Across all age groups, the most common age male respondants started coding was between ages 11-17. The second most occurring age range for developing interest in coding amongst men is 18-24 years. <br> 
Comparing these results with female survey respondents, the difference between start of coding at 11-17 years and at 18-24 years is not as pronounced. **This indicates that, compared to men, women are less likely to start coding earlier in life. Interestingly, results for non-binary, gender queer, transgender and non-conforming respondants are more similar to those of men than women.** 

It would be interesting to further investigate the noticed trend and what might be the reason that women are more likely to pick up coding at an older age than men or NB, GQ and TGNC. The questions that arose from this section are:
 -  Are young girls discouraged from expressing interest in programming because boys still make up majority of programming classes/groups (if we were to extrapolate on the findings from the first section)?
 -  Is it because of some underlying societal reason?
 -  How come NB, GQ and TGNC individuals do not seem to be affected by the same reasons?

#### Appendix

Table comparison for some of the other survey questions.

In [38]:
gnd_0_perc = counter(gnd_0, 'Age1stCode')
gnd_m_perc = counter(gnd_m, 'Age1stCode')
gnd_w_perc = counter(gnd_w, 'Age1stCode')
gnd_nb_perc = counter(gnd_nb, 'Age1stCode')

comp_df = pd.merge(gnd_m_perc, gnd_w_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_nb_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_0_perc, left_on='answer', right_on='answer')

sorter=[ 'Younger than 5 years','5 - 10 years','11 - 17 years','18 - 24 years','25 - 34 years', '35 - 44 years', '45 - 54 years', '55 - 64 years', 'Older than 64 years']
sorterIndex = dict(zip(sorter, range(len(sorter))))

comp_df.columns = ['Age1stCode','Men', 'Women', 'Nonbinary', 'NotDisclosed']
comp_df['Age1stCode_Rank'] = comp_df['Age1stCode'].map(sorterIndex)

comp_df['GenderDiff_MW'] = comp_df['Men'] - comp_df['Women']
comp_df['GenderDiff_MNb'] = comp_df['Men'] - comp_df['Nonbinary']
comp_df['GenderDiff_MND'] = comp_df['Men'] - comp_df['NotDisclosed']
comp_df.sort_values(['Age1stCode_Rank'],ascending = True, inplace = True)
comp_df.drop('Age1stCode_Rank', 1, inplace = True)
comp_df.reset_index(inplace=True, drop=True)

#comp_df.sort_values(by=['GenderDiff_MW'], ignore_index=True, inplace=True, ascending=False)
comp_df.style.bar(subset=['GenderDiff_MW'], align='mid', color=['#fec20c', '#00ab41']).bar(subset=['GenderDiff_MNb'], align='mid', color=['#6a0dad', '#00ab41']).bar(subset=['GenderDiff_MND'], align='mid', color=['#8e8e8e', '#00ab41']).format({'Men': '{:,.2%}'.format,
    'Women': '{:,.2%}'.format,'Nonbinary': '{:,.2%}'.format,'NotDisclosed': '{:,.2%}'.format,'GenderDiff_MW': '{:,.2%}'.format,'GenderDiff_MNb': '{:,.2%}'.format,'GenderDiff_MND': '{:,.2%}'.format,
})

In [39]:
gnd_0_perc = counter(gnd_0, 'YearsCodeGrouped')
gnd_m_perc = counter(gnd_m, 'YearsCodeGrouped')
gnd_w_perc = counter(gnd_w, 'YearsCodeGrouped')
gnd_nb_perc = counter(gnd_nb, 'YearsCodeGrouped')

comp_df = pd.merge(gnd_m_perc, gnd_w_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_nb_perc, left_on='answer', right_on='answer')
comp_df = pd.merge(comp_df, gnd_0_perc, left_on='answer', right_on='answer')

sorter=['Less than 1 year','1 - 5 years', '6 - 10 years','11 - 20 years', '21 - 50 years','More than 50 years']
sorterIndex = dict(zip(sorter, range(len(sorter))))

comp_df.columns = ['YearsCodeGrouped','Men', 'Women', 'Nonbinary', 'NotDisclosed']
comp_df['YearsCodeGrouped_Rank'] = comp_df['YearsCodeGrouped'].map(sorterIndex)

comp_df['GenderDiff_MW'] = comp_df['Men'] - comp_df['Women']
comp_df['GenderDiff_MNb'] = comp_df['Men'] - comp_df['Nonbinary']
comp_df['GenderDiff_MND'] = comp_df['Men'] - comp_df['NotDisclosed']
comp_df.sort_values(['YearsCodeGrouped_Rank'],ascending = True, inplace = True)
comp_df.drop('YearsCodeGrouped_Rank', 1, inplace = True)
comp_df.reset_index(inplace=True, drop=True)

#comp_df.sort_values(by=['GenderDiff_MW'], ignore_index=True, inplace=True, ascending=False)
comp_df.style.bar(subset=['GenderDiff_MW'], align='mid', color=['#fec20c', '#00ab41']).bar(subset=['GenderDiff_MNb'], align='mid', color=['#6a0dad', '#00ab41']).bar(subset=['GenderDiff_MND'], align='mid', color=['#8e8e8e', '#00ab41']).format({'Men': '{:,.2%}'.format,
    'Women': '{:,.2%}'.format,'Nonbinary': '{:,.2%}'.format,'NotDisclosed': '{:,.2%}'.format,'GenderDiff_MW': '{:,.2%}'.format,'GenderDiff_MNb': '{:,.2%}'.format,'GenderDiff_MND': '{:,.2%}'.format,
})

In [40]:
list(df.Age.value_counts().index)

['25-34 years old',
 '18-24 years old',
 '35-44 years old',
 '45-54 years old',
 'Under 18 years old',
 '55-64 years old',
 'Prefer not to say',
 '65 years or older']